# Youtube Video Analysis



In [1]:
# install kaggle api and set the credentials
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:

!kaggle datasets download -d datasnaek/youtube-new
!mv youtube-new.zip input/
!mkdir -p input/youtube-new
!unzip input/youtube-new.zip -d input/youtube-new

Dataset URL: https://www.kaggle.com/datasets/datasnaek/youtube-new
License(s): CC0-1.0
100%|████████████████████████████████████████▊| 200M/201M [00:00<00:00, 601MB/s]
100%|█████████████████████████████████████████| 201M/201M [00:00<00:00, 641MB/s]
Archive:  input/youtube-new.zip
  inflating: input/youtube-new/CA_category_id.json  
  inflating: input/youtube-new/CAvideos.csv  
  inflating: input/youtube-new/DE_category_id.json  
  inflating: input/youtube-new/DEvideos.csv  
  inflating: input/youtube-new/FR_category_id.json  
  inflating: input/youtube-new/FRvideos.csv  
  inflating: input/youtube-new/GB_category_id.json  
  inflating: input/youtube-new/GBvideos.csv  
  inflating: input/youtube-new/IN_category_id.json  
  inflating: input/youtube-new/INvideos.csv  
  inflating: input/youtube-new/JP_category_id.json  
  inflating: input/youtube-new/JPvideos.csv  
  inflating: input/youtube-new/KR_category_id.json  
  inflating: input/youtube-new/KRvideos.csv  
  inflating: input/youtube

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import json

# Input data files are available in the read-only "../input/" directory
import os

from helpers.utils import dataframe_summary

In [4]:
# Define Constants
DATASET_DIR = "input/youtube-new"

In [5]:
for dirname, _, filenames in os.walk(DATASET_DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))

input/youtube-new/RUvideos.csv
input/youtube-new/KRvideos.csv
input/youtube-new/INvideos.csv
input/youtube-new/USvideos.csv
input/youtube-new/DE_category_id.json
input/youtube-new/JPvideos.csv
input/youtube-new/CA_category_id.json
input/youtube-new/KR_category_id.json
input/youtube-new/IN_category_id.json
input/youtube-new/FR_category_id.json
input/youtube-new/MX_category_id.json
input/youtube-new/JP_category_id.json
input/youtube-new/CAvideos.csv
input/youtube-new/DEvideos.csv
input/youtube-new/GBvideos.csv
input/youtube-new/FRvideos.csv
input/youtube-new/GB_category_id.json
input/youtube-new/RU_category_id.json
input/youtube-new/MXvideos.csv
input/youtube-new/US_category_id.json


## Data Preparation

- Merge `input/youtube-new/**videos.csv` into a single dataframe (videos_df).
- Merge `input/youtube-new/**_category_id.json` into a single dataframe (categories_df).

In [6]:
## Loading the videos files into a single frame
videos_df_list = []
for filename in glob.glob(os.path.join(DATASET_DIR, "**videos.csv")):
    print("Loading file {}...".format(filename))
    video_df = pd.read_csv(filename, encoding='latin-1')
    video_df["filename"] = filename
    videos_df_list.append(video_df)
    print("Loaded file {}.".format(filename))

videos_df = pd.concat(videos_df_list, ignore_index=True)

Loading file input/youtube-new/RUvideos.csv...
Loaded file input/youtube-new/RUvideos.csv.
Loading file input/youtube-new/KRvideos.csv...
Loaded file input/youtube-new/KRvideos.csv.
Loading file input/youtube-new/INvideos.csv...
Loaded file input/youtube-new/INvideos.csv.
Loading file input/youtube-new/USvideos.csv...
Loaded file input/youtube-new/USvideos.csv.
Loading file input/youtube-new/JPvideos.csv...
Loaded file input/youtube-new/JPvideos.csv.
Loading file input/youtube-new/CAvideos.csv...
Loaded file input/youtube-new/CAvideos.csv.
Loading file input/youtube-new/DEvideos.csv...
Loaded file input/youtube-new/DEvideos.csv.
Loading file input/youtube-new/GBvideos.csv...
Loaded file input/youtube-new/GBvideos.csv.
Loading file input/youtube-new/FRvideos.csv...
Loaded file input/youtube-new/FRvideos.csv.
Loading file input/youtube-new/MXvideos.csv...
Loaded file input/youtube-new/MXvideos.csv.


In [7]:
## Loading the categories files into a single dataframe
categories_df_list = []
for filename in glob.glob(os.path.join(DATASET_DIR, "**_category_id.json")):
    print("Loading file {}...".format(filename))
    category_data = json.load(open(filename, "r"))
    category_df = pd.json_normalize(category_data, record_path="items", sep="_")
    category_df["filename"] = filename
    categories_df_list.append(category_df)

categories_df = pd.concat(categories_df_list, ignore_index=True)

Loading file input/youtube-new/DE_category_id.json...
Loading file input/youtube-new/CA_category_id.json...
Loading file input/youtube-new/KR_category_id.json...
Loading file input/youtube-new/IN_category_id.json...
Loading file input/youtube-new/FR_category_id.json...
Loading file input/youtube-new/MX_category_id.json...
Loading file input/youtube-new/JP_category_id.json...
Loading file input/youtube-new/GB_category_id.json...
Loading file input/youtube-new/RU_category_id.json...
Loading file input/youtube-new/US_category_id.json...


## Data Preprocessing

- Extract the `country_cd` from the `filename`.
- Convert the `category_id` column into `int`.
- Convert the `trending_date` into datetime

In [8]:
# Extracting the country from the filename
videos_df['country_cd'] = videos_df.filename.str.extract("new/([A-Z]+)videos.csv")
print("Unique countries: {}".format(list(videos_df.country_cd.unique())))

categories_df['country_cd'] = categories_df.filename.str.extract("new/([A-Z]+)_category_id.json")
print("Unique countries: {}".format(list(categories_df.country_cd.unique())))

Unique countries: ['RU', 'KR', 'IN', 'US', 'JP', 'CA', 'DE', 'GB', 'FR', 'MX']
Unique countries: ['DE', 'CA', 'KR', 'IN', 'FR', 'MX', 'JP', 'GB', 'RU', 'US']


In [9]:
# Convert the category_id to int
categories_df["id"] = categories_df["id"].astype("int")
videos_df["category_id"] = videos_df["category_id"].astype("int")

In [10]:
# Convert the trending date into ISO format
videos_df["trending_date"] = pd.to_datetime(videos_df["trending_date"], format="%y.%d.%m")

In [11]:
videos_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,filename,country_cd
0,gDuslQ9avLc,2017-11-14,ÐÐ°Ñ Ð°Ñ Ð¸ ÐÐ¾Ð»Ð¸Ð½Ð° ÑÑÐ°ÑÑÑ ÑÐºÐ¾...,Ð¢âÐ ÐÐÐÐÐ§,22,2017-11-13T09:09:31.000Z,"Ð·Ð°Ñ Ð°Ñ Ð¸ Ð¿Ð¾Ð»Ð¸Ð½Ð°|""ÑÑÐ¸Ð¼ÑÑ ÑÐºÐ...",62408,334,190,50,https://i.ytimg.com/vi/gDuslQ9avLc/default.jpg,False,False,False,"ÐÐ½Ð°ÐºÐ¾Ð¼ÑÑÐµÑÑ, ÑÑÐ¾ ÐÐ°Ñ Ð°Ñ Ð¸ Ð...",input/youtube-new/RUvideos.csv,RU
1,AOCJIFEA_jE,2017-11-14,ÐÐ¸ÑÐ¶Ð° ÐÐµÐ¼Ð¾Ð² #29. ÐÐ¾Ð»ÑÑÐ¾Ð¸Ì Ð²...,Druzhko Show,22,2017-11-13T17:32:11.000Z,"Ð±Ð¸ÑÐ¶Ð° Ð¼ÐµÐ¼Ð¾Ð²|""Ð»ÐµÐ² ÑÐ°Ð³Ð¸Ð½ÑÐ½""|...",330043,43841,2244,2977,https://i.ytimg.com/vi/AOCJIFEA_jE/default.jpg,False,False,False,Ð 29 Ð²ÑÐ¿ÑÑÐºÐµ ÐÑÑÐ¶ÐºÐ¾ Ð¨Ð¾Ñ Ð¡ÐµÑ...,input/youtube-new/RUvideos.csv,RU
2,VAWNQDgwwOM,2017-11-14,Ð¥ÐÐÐ ÐÐ­ÐÐ - Ð¡ÐÐÐ Ð¡Ð ÐÐÐ Ð§Ð£Ð...,Ð®Ð»Ð¸Ðº,24,2017-11-13T16:11:31.000Z,"ÑÐ¼Ð¾Ñ|""ÐºÐ¾Ð¼ÐµÐ´Ð¸Ñ""|""Ð²Ð»Ð¾Ð³""|""Ð±Ð»Ð¾Ð³...",424596,49854,714,2944,https://i.ytimg.com/vi/VAWNQDgwwOM/default.jpg,False,False,False,http://kapitany.ru/!Yulik.cap - Ð¤Ð°ÐºÑÐ»ÑÑ...,input/youtube-new/RUvideos.csv,RU
3,gknkFwKQfHg,2017-11-14,Ð¡Ð¾ÑÐ½Ð°Ñ ÐºÐµÑÐ°Ð´Ð¸Ð»ÑÑ Ñ ÐºÑÑÐ¸ÑÐµÐ¹,Hochland,22,2017-11-13T06:51:10.000Z,"Ñ Ð¾Ñ Ð»Ð°Ð½Ð´|""ÑÑÑ""|""ÑÐµÑÐµÐ¿ÑÑ""|""ÐºÐ°...",112851,3566,122,80,https://i.ytimg.com/vi/gknkFwKQfHg/default.jpg,False,False,False,*** ÐºÐ°Ðº Ð³Ð¾ÑÐ¾Ð²Ð¸ÑÑ ÑÐµÑÑÐ¾ÑÐ°Ð½Ð½...,input/youtube-new/RUvideos.csv,RU
4,3sYvZcwzZr8,2017-11-14,ÐÐÐÐÐ« Ð ÐÐÐÐ¢ÐÐÐÐ ÐÐ Ð¨ÐÐÐÐ¬Ð...,Ð¡Ð¾Ð²ÐµÑÐ³Ð¾Ð½,24,2017-11-13T16:52:36.000Z,"Ð¡Ð¾Ð²ÐµÑÐ³Ð¾Ð½|""Sovergon""|""ÐºÐ»Ð¸Ð¿Ñ""|""ÑÐ¾...",243469,36216,631,1692,https://i.ytimg.com/vi/3sYvZcwzZr8/default.jpg,False,False,False,â ÐÐÐÐÐ£Ð Ð¡ ÐÐÐÐÐ¢ÐÐÐ«: http://ka...,input/youtube-new/RUvideos.csv,RU


In [12]:
dataframe_summary(videos_df)

## DataFrame Summary
---------------
**Shape**: (375942, 18)  
**Columns**: ['video_id', 'trending_date', 'title', 'channel_title', 'category_id', 'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed', 'description', 'filename', 'country_cd']  
### Column Summary:
|Column|Type|Missing|Unique|Min|Max|Mean|
|---|---|---|---|---|---|---|
|video_id|object|0|184287||||
|trending_date|datetime64[ns]|0|205|.5g|.5g|.5g|
|title|object|0|186272||||
|channel_title|object|0|37824||||
|category_id|int64|0|18|1|44|20.232|
|publish_time|object|0|169286||||
|tags|object|0|142211||||
|views|int64|0|240399|117|4.2454e+08|1.3266e+06|
|likes|int64|0|73693|0|5.6138e+06|37884|
|dislikes|int64|0|16715|0|1.945e+06|2126.1|
|comment_count|int64|0|24477|0|1.6265e+06|4253.8|
|thumbnail_link|object|0|185690||||
|comments_disabled|bool|0|2||||
|ratings_disabled|bool|0|2||||
|video_error_or_removed|bool|0|2||||
|description|object|19478|156165||||
|filename|object|0|10||||
|country_cd|object|0|10||||


In [13]:
categories_df.head()

,kind,etag,id,snippet_channelId,snippet_title,snippet_assignable,filename,country_cd
0,youtube#videoCategory,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/Xy1mB4_yLrHy_BmKm...",1,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation,True,input/youtube-new/DE_category_id.json,DE
1,youtube#videoCategory,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/UZ1oLIIz2dxIhO45Z...",2,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles,True,input/youtube-new/DE_category_id.json,DE
2,youtube#videoCategory,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/nqRIq97-xe5XRZTxb...",10,UCBR8-60-B28hp2BmDPdntcQ,Music,True,input/youtube-new/DE_category_id.json,DE
3,youtube#videoCategory,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/HwXKamM1Q20q9BN-o...",15,UCBR8-60-B28hp2BmDPdntcQ,Pets & Animals,True,input/youtube-new/DE_category_id.json,DE
4,youtube#videoCategory,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/9GQMSRjrZdHeb1OEM...",17,UCBR8-60-B28hp2BmDPdntcQ,Sports,True,input/youtube-new/DE_category_id.json,DE


In [14]:
dataframe_summary(categories_df)

## DataFrame Summary
---------------
**Shape**: (311, 8)  
**Columns**: ['kind', 'etag', 'id', 'snippet_channelId', 'snippet_title', 'snippet_assignable', 'filename', 'country_cd']  
### Column Summary:
|Column|Type|Missing|Unique|Min|Max|Mean|
|---|---|---|---|---|---|---|
|kind|object|0|1||||
|etag|object|0|94||||
|id|int64|0|32|1|44|27.521|
|snippet_channelId|object|0|1||||
|snippet_title|object|0|31||||
|snippet_assignable|bool|0|2||||
|filename|object|0|10||||
|country_cd|object|0|10||||


In [15]:
# Drop the unnecessary columns
videos_df = videos_df.drop(["filename", "video_id"], axis=1)
categories_df = categories_df.drop(["filename", "kind", "etag", "snippet_channelId"], axis=1)

In [16]:
# Merge the dataframes
videos_df = videos_df.merge(categories_df, how="left", left_on=["category_id", "country_cd"], right_on=["id", "country_cd"])
videos_df.head()

,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country_cd,id,snippet_title,snippet_assignable
0,2017-11-14,ÐÐ°Ñ Ð°Ñ Ð¸ ÐÐ¾Ð»Ð¸Ð½Ð° ÑÑÐ°ÑÑÑ ÑÐºÐ¾...,Ð¢âÐ ÐÐÐÐÐ§,22,2017-11-13T09:09:31.000Z,"Ð·Ð°Ñ Ð°Ñ Ð¸ Ð¿Ð¾Ð»Ð¸Ð½Ð°|""ÑÑÐ¸Ð¼ÑÑ ÑÐºÐ...",62408,334,190,50,https://i.ytimg.com/vi/gDuslQ9avLc/default.jpg,False,False,False,"ÐÐ½Ð°ÐºÐ¾Ð¼ÑÑÐµÑÑ, ÑÑÐ¾ ÐÐ°Ñ Ð°Ñ Ð¸ Ð...",RU,22.0,People & Blogs,True
1,2017-11-14,ÐÐ¸ÑÐ¶Ð° ÐÐµÐ¼Ð¾Ð² #29. ÐÐ¾Ð»ÑÑÐ¾Ð¸Ì Ð²...,Druzhko Show,22,2017-11-13T17:32:11.000Z,"Ð±Ð¸ÑÐ¶Ð° Ð¼ÐµÐ¼Ð¾Ð²|""Ð»ÐµÐ² ÑÐ°Ð³Ð¸Ð½ÑÐ½""|...",330043,43841,2244,2977,https://i.ytimg.com/vi/AOCJIFEA_jE/default.jpg,False,False,False,Ð 29 Ð²ÑÐ¿ÑÑÐºÐµ ÐÑÑÐ¶ÐºÐ¾ Ð¨Ð¾Ñ Ð¡ÐµÑ...,RU,22.0,People & Blogs,True
2,2017-11-14,Ð¥ÐÐÐ ÐÐ­ÐÐ - Ð¡ÐÐÐ Ð¡Ð ÐÐÐ Ð§Ð£Ð...,Ð®Ð»Ð¸Ðº,24,2017-11-13T16:11:31.000Z,"ÑÐ¼Ð¾Ñ|""ÐºÐ¾Ð¼ÐµÐ´Ð¸Ñ""|""Ð²Ð»Ð¾Ð³""|""Ð±Ð»Ð¾Ð³...",424596,49854,714,2944,https://i.ytimg.com/vi/VAWNQDgwwOM/default.jpg,False,False,False,http://kapitany.ru/!Yulik.cap - Ð¤Ð°ÐºÑÐ»ÑÑ...,RU,24.0,Entertainment,True
3,2017-11-14,Ð¡Ð¾ÑÐ½Ð°Ñ ÐºÐµÑÐ°Ð´Ð¸Ð»ÑÑ Ñ ÐºÑÑÐ¸ÑÐµÐ¹,Hochland,22,2017-11-13T06:51:10.000Z,"Ñ Ð¾Ñ Ð»Ð°Ð½Ð´|""ÑÑÑ""|""ÑÐµÑÐµÐ¿ÑÑ""|""ÐºÐ°...",112851,3566,122,80,https://i.ytimg.com/vi/gknkFwKQfHg/default.jpg,False,False,False,*** ÐºÐ°Ðº Ð³Ð¾ÑÐ¾Ð²Ð¸ÑÑ ÑÐµÑÑÐ¾ÑÐ°Ð½Ð½...,RU,22.0,People & Blogs,True
4,2017-11-14,ÐÐÐÐÐ« Ð ÐÐÐÐ¢ÐÐÐÐ ÐÐ Ð¨ÐÐÐÐ¬Ð...,Ð¡Ð¾Ð²ÐµÑÐ³Ð¾Ð½,24,2017-11-13T16:52:36.000Z,"Ð¡Ð¾Ð²ÐµÑÐ³Ð¾Ð½|""Sovergon""|""ÐºÐ»Ð¸Ð¿Ñ""|""ÑÐ¾...",243469,36216,631,1692,https://i.ytimg.com/vi/3sYvZcwzZr8/default.jpg,False,False,False,â ÐÐÐÐÐ£Ð Ð¡ ÐÐÐÐÐ¢ÐÐÐ«: http://ka...,RU,24.0,Entertainment,True


In [17]:
# Drop more unnecessary columns
videos_df = videos_df.drop(["id", "category_id"], axis=1)

In [18]:
dataframe_summary(videos_df)

## DataFrame Summary
---------------
**Shape**: (375942, 17)  
**Columns**: ['trending_date', 'title', 'channel_title', 'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed', 'description', 'country_cd', 'snippet_title', 'snippet_assignable']  
### Column Summary:
|Column|Type|Missing|Unique|Min|Max|Mean|
|---|---|---|---|---|---|---|
|trending_date|datetime64[ns]|0|205|.5g|.5g|.5g|
|title|object|0|186272||||
|channel_title|object|0|37824||||
|publish_time|object|0|169286||||
|tags|object|0|142211||||
|views|int64|0|240399|117|4.2454e+08|1.3266e+06|
|likes|int64|0|73693|0|5.6138e+06|37884|
|dislikes|int64|0|16715|0|1.945e+06|2126.1|
|comment_count|int64|0|24477|0|1.6265e+06|4253.8|
|thumbnail_link|object|0|185690||||
|comments_disabled|bool|0|2||||
|ratings_disabled|bool|0|2||||
|video_error_or_removed|bool|0|2||||
|description|object|19478|156165||||
|country_cd|object|0|10||||
|snippet_title|object|2738|18||||
|snippet_assignable|object|2738|2||||


In [19]:
videos_df.head()

,trending_date,title,channel_title,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country_cd,snippet_title,snippet_assignable
0,2017-11-14,ÐÐ°Ñ Ð°Ñ Ð¸ ÐÐ¾Ð»Ð¸Ð½Ð° ÑÑÐ°ÑÑÑ ÑÐºÐ¾...,Ð¢âÐ ÐÐÐÐÐ§,2017-11-13T09:09:31.000Z,"Ð·Ð°Ñ Ð°Ñ Ð¸ Ð¿Ð¾Ð»Ð¸Ð½Ð°|""ÑÑÐ¸Ð¼ÑÑ ÑÐºÐ...",62408,334,190,50,https://i.ytimg.com/vi/gDuslQ9avLc/default.jpg,False,False,False,"ÐÐ½Ð°ÐºÐ¾Ð¼ÑÑÐµÑÑ, ÑÑÐ¾ ÐÐ°Ñ Ð°Ñ Ð¸ Ð...",RU,People & Blogs,True
1,2017-11-14,ÐÐ¸ÑÐ¶Ð° ÐÐµÐ¼Ð¾Ð² #29. ÐÐ¾Ð»ÑÑÐ¾Ð¸Ì Ð²...,Druzhko Show,2017-11-13T17:32:11.000Z,"Ð±Ð¸ÑÐ¶Ð° Ð¼ÐµÐ¼Ð¾Ð²|""Ð»ÐµÐ² ÑÐ°Ð³Ð¸Ð½ÑÐ½""|...",330043,43841,2244,2977,https://i.ytimg.com/vi/AOCJIFEA_jE/default.jpg,False,False,False,Ð 29 Ð²ÑÐ¿ÑÑÐºÐµ ÐÑÑÐ¶ÐºÐ¾ Ð¨Ð¾Ñ Ð¡ÐµÑ...,RU,People & Blogs,True
2,2017-11-14,Ð¥ÐÐÐ ÐÐ­ÐÐ - Ð¡ÐÐÐ Ð¡Ð ÐÐÐ Ð§Ð£Ð...,Ð®Ð»Ð¸Ðº,2017-11-13T16:11:31.000Z,"ÑÐ¼Ð¾Ñ|""ÐºÐ¾Ð¼ÐµÐ´Ð¸Ñ""|""Ð²Ð»Ð¾Ð³""|""Ð±Ð»Ð¾Ð³...",424596,49854,714,2944,https://i.ytimg.com/vi/VAWNQDgwwOM/default.jpg,False,False,False,http://kapitany.ru/!Yulik.cap - Ð¤Ð°ÐºÑÐ»ÑÑ...,RU,Entertainment,True
3,2017-11-14,Ð¡Ð¾ÑÐ½Ð°Ñ ÐºÐµÑÐ°Ð´Ð¸Ð»ÑÑ Ñ ÐºÑÑÐ¸ÑÐµÐ¹,Hochland,2017-11-13T06:51:10.000Z,"Ñ Ð¾Ñ Ð»Ð°Ð½Ð´|""ÑÑÑ""|""ÑÐµÑÐµÐ¿ÑÑ""|""ÐºÐ°...",112851,3566,122,80,https://i.ytimg.com/vi/gknkFwKQfHg/default.jpg,False,False,False,*** ÐºÐ°Ðº Ð³Ð¾ÑÐ¾Ð²Ð¸ÑÑ ÑÐµÑÑÐ¾ÑÐ°Ð½Ð½...,RU,People & Blogs,True
4,2017-11-14,ÐÐÐÐÐ« Ð ÐÐÐÐ¢ÐÐÐÐ ÐÐ Ð¨ÐÐÐÐ¬Ð...,Ð¡Ð¾Ð²ÐµÑÐ³Ð¾Ð½,2017-11-13T16:52:36.000Z,"Ð¡Ð¾Ð²ÐµÑÐ³Ð¾Ð½|""Sovergon""|""ÐºÐ»Ð¸Ð¿Ñ""|""ÑÐ¾...",243469,36216,631,1692,https://i.ytimg.com/vi/3sYvZcwzZr8/default.jpg,False,False,False,â ÐÐÐÐÐ£Ð Ð¡ ÐÐÐÐÐ¢ÐÐÐ«: http://ka...,RU,Entertainment,True


In [21]:
videos_df.to_csv("output/videos.csv", index=False)